In [1]:
import xarray_beam as xb

ds, _ = xb.open_zarr('gcs://era5_climate/1440x720/2011-12-31_2021-12-31.zarr')

In [2]:
print(ds.nbytes / (10**9))

73.598609492


In [67]:
temp = ds['2m_temperature']

In [5]:
import numpy as np
import pandas as pd

lat_min, lat_max = (15, 45.0)
lon_min, lon_max = (115, 145)

lat_indices = np.where((ds.latitude >= lat_min) & (ds.latitude <= lat_max))[0]
lon_indices = np.where((ds.longitude >= lon_min) & (ds.longitude <= lon_max))[0]

source_dataset = ds.isel(latitude=lat_indices, longitude=lon_indices)

start_date = pd.to_datetime('2020-12-28')
end_date = pd.to_datetime('2021-12-29')
source_dataset = source_dataset.sel(time=slice(start_date, end_date))

print(source_dataset)



<xarray.Dataset>
Dimensions:                       (time: 1465, latitude: 121, longitude: 121,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float32 45.0 44.75 ... 15.25 15.0
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float32 115.0 115.2 ... 145.0
  * time                          (time) datetime64[ns] 2020-12-28 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, latitude, longitude) float32 ...
    10m_v_component_of_wind       (time, latitude, longitude) float32 ...
    2m_temperature                (time, latitude, longitude) float32 ...
    geopotential                  (time, level, latitude, longitude) float32 ...
    land_sea_mask                 (latitude, longitude) float32 ...
    mean_sea_level_pressure       (time, latitude, longitude) float32 ...
    ...                            ...
    toa_incident_solar

In [6]:
print(source_dataset.nbytes / (10**9))

7.378549716


In [12]:
dataset = source_dataset['2m_temperature'].values

In [13]:
print(dataset.shape)

(1465, 121, 121)


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# 예제 NumPy 배열 생성
data = dataset[0][::-1, :]
print(data.shape)

data = data - 273.15

# 지도 생성 및 데이터 시각화
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# 지도에 기본 레이어 추가
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# NumPy 배열을 지도 위에 표시 (임의의 위경도 범위에 맞게 조정)
lons = np.linspace(lon_min, lon_max, data.shape[1])
lats = np.linspace(lat_min, lat_max, data.shape[0])
Lons, Lats = np.meshgrid(lons, lats)
mesh = ax.pcolormesh(Lons, Lats, data, transform=ccrs.PlateCarree())

plt.colorbar(mesh, ax=ax, shrink=0.5, aspect=5)

plt.show()

TypeError: 'method' object is not subscriptable

In [25]:
import torch

def get_tgt_mask(var_len, tgt_time_len) -> torch.tensor:
        matrix = torch.zeros(var_len * tgt_time_len, var_len * tgt_time_len)

        for i in range(tgt_time_len):
            for j in range(var_len):
                inf_idx = min(((i)*var_len), var_len * tgt_time_len)
                matrix[:(i*var_len), inf_idx:] = float('-inf')
        return matrix


print(get_tgt_mask(5, 5))

tensor([[0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -